# Searching for Simpson

Copyright 2021 Allen B. Downey

License: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

[Click here to run this notebook on Colab](https://colab.research.google.com/github/AllenDowney/ProbablyOverthinkingIt2/blob/master/simpson_wages.ipynb)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
gss = pd.read_hdf('gss_eda.3.hdf5', 'gss0')
gss.shape

(64814, 169)

In [3]:
recode_polviews = {1:'Liberal', 
                   2:'Liberal', 
                   3:'Liberal', 
                   4:'Moderate', 
                   5:'Conservative', 
                   6:'Conservative', 
                   7:'Conservative'}

In [4]:
gss['polviews3'] = gss['polviews'].replace(recode_polviews)
gss['polviews3'].value_counts()

Moderate        21444
Conservative    19129
Liberal         14979
Name: polviews3, dtype: int64

>Generally speaking, do you usually think of yourself as a Republican, Democrat, Independent, or what?

The valid responses are:

```
0	Strong democrat
1	Not str democrat
2	Ind,near dem
3	Independent
4	Ind,near rep
5	Not str republican
6	Strong republican
7	Other party
```

You can [read the codebook for `partyid` here](https://gssdataexplorer.norc.org/projects/52787/variables/141/vshow).

In [5]:
recode_partyid = {0: 'Democrat',
                  1:'Democrat', 
                   2:'Independent', 
                   3:'Independent', 
                   4:'Independent', 
                   5:'Republican', 
                   6:'Republican', 
                   7:'Other'}

In [6]:
gss['partyid4'] = gss['partyid'].replace(recode_partyid)
gss['partyid4'].value_counts()

Independent    23404
Democrat       23308
Republican     16617
Other           1064
Name: partyid4, dtype: int64

Respondent's highest degree

```
0 	Lt high school
1 	High school
2 	Junior college
3 	Bachelor
4 	Graduate
8 	Don't know
9 	No answer
```



In [7]:
gss['degree'].value_counts()

1.0    33855
0.0    13274
3.0     9277
4.0     4465
2.0     3759
Name: degree, dtype: int64

> What is your religious preference? Is it Protestant, Catholic, Jewish, some other religion, or no religion?

```
1 	Protestant
2 	Catholic
3 	Jewish
4 	None
5 	Other
6 	Buddhism
7 	Hinduism
8 	Other eastern
9 	Moslem/islam
10 	Orthodox-christian
11 	Christian
12 	Native american
13 	Inter-nondenominational
```



In [8]:
recode_relig = {1:'Protestant', 
                   2:'Catholic', 
                   3:'Other', 
                   4:'None', 
                   5:'Other', 
                   6:'Other', 
                   7:'Other', 
                   8:'Other', 
                   9:'Other', 
                   10:'Other Christian', 
                   11:'Other Christian', 
                   12:'Other', 
                   13:'Other'}

In [9]:
gss['relig5'] = gss['relig'].replace(recode_relig)
gss['relig5'].value_counts()

Protestant         36378
Catholic           16501
None                7803
Other               2966
Other Christian      896
Name: relig5, dtype: int64

> If you were asked to use one of four names for your social class, which would you say you belong in: the lower class, the working class, the middle class, or the upper class?
 
```
1 	Lower class
2 	Working class
3 	Middle class
4 	Upper class
5 	No class
8 	Don't know
9 	No answer
0 	Not applicable
```

In [10]:
gss['class_'].value_counts()

2.0    28215
3.0    27746
1.0     3398
4.0     1969
Name: class_, dtype: int64

In [11]:
gss['age'].head()

0    60.0
1    53.0
2    72.0
3    19.0
4    44.0
Name: age, dtype: float32

In [12]:
bins = np.arange(17, 95, 5)
labels = bins[:-1] + 3

gss['age5'] = pd.cut(gss['age'], bins, labels=labels)
gss['age5'].head()

0    60
1    55
2    70
3    20
4    45
Name: age5, dtype: category
Categories (15, int64): [20 < 25 < 30 < 35 ... 75 < 80 < 85 < 90]

In [13]:
gss['age5'].value_counts().sort_index()

20    5308
25    7010
30    6776
35    6534
40    6176
45    6103
50    5763
55    5153
60    4457
65    3719
70    3044
75    2155
80    1329
85     731
90     375
Name: age5, dtype: int64

In [14]:
gss['cohort'].head()

0    1912.0
1    1919.0
2    1900.0
3    1953.0
4    1928.0
Name: cohort, dtype: float32

In [15]:
bins = np.arange(1889, 2001, 10)
labels = bins[:-1] + 1

gss['cohort10'] = pd.cut(gss['cohort'], bins, labels=labels)
gss['cohort10'].head()

0    1910
1    1910
2    1900
3    1950
4    1920
Name: cohort10, dtype: category
Categories (11, int64): [1890 < 1900 < 1910 < 1920 ... 1960 < 1970 < 1980 < 1990]

In [16]:
gss['cohort10'].value_counts().sort_index()

1890      455
1900     1717
1910     3663
1920     5959
1930     6889
1940    10463
1950    13422
1960    10123
1970     6705
1980     3821
1990     1345
Name: cohort10, dtype: int64

In [17]:
gss['year'].tail()

64809    2018
64810    2018
64811    2018
64812    2018
64813    2018
Name: year, dtype: int64

In [18]:
bins = np.arange(1970, 2025, 5)
bins

array([1970, 1975, 1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015, 2020])

In [19]:
labels = bins[:-1] + 2

gss['year5'] = pd.cut(gss['year'], bins, labels=labels)
gss['year5'].tail()

64809    2017
64810    2017
64811    2017
64812    2017
64813    2017
Name: year5, dtype: category
Categories (10, int64): [1972 < 1977 < 1982 < 1987 ... 2002 < 2007 < 2012 < 2017]

In [20]:
gss['year5'].value_counts().sort_index()

1972    6091
1977    6029
1982    6466
1987    7679
1992    6115
1997    8553
2002    5577
2007    8577
2012    4512
2017    5215
Name: year5, dtype: int64

## Run Subgroups

In [180]:
xvarname = 'year'
yvarname = 'fund'
gvarname = 'relig5'

In [181]:
yvar = gss[yvarname]

In [182]:
counts = yvar.value_counts()
counts

2.0    27000
1.0    18789
3.0    16513
Name: fund, dtype: int64

In [183]:
most_common = counts.idxmax()
most_common

2.0

In [184]:
d = counts.copy()
d[:] = 0
d[most_common] = 1
d

2.0    1
1.0    0
3.0    0
Name: fund, dtype: int64

In [185]:
gss['y'] = yvar.replace(d)
gss['y'].value_counts()

0.0    35302
1.0    27000
Name: y, dtype: int64

In [186]:
gss['y'].isnull().sum()

2512

In [187]:
gss['x'] = gss[xvarname]

In [188]:
import statsmodels.formula.api as smf

formula = 'y ~ x'
results = smf.logit(formula, data=gss).fit()

Optimization terminated successfully.
         Current function value: 0.682015
         Iterations 4


In [189]:
param = results.params['x']
param

-0.010007568995860933

In [190]:
pvalue = results.pvalues['x']
pvalue

4.794654437207781e-62

In [191]:
conf_int = results.conf_int().loc['x'].values
conf_int

array([-0.01118757, -0.00882757])

In [192]:
stderr = results.bse['x']

In [193]:
def get_xresult(results):
    param = results.params['x']
    pvalue = results.pvalues['x']
    conf_int = results.conf_int().loc['x'].values
    stderr = results.bse['x']
    return [param, pvalue, stderr, conf_int]

In [194]:
columns = ['param', 'pvalue', 'stderr', 'conf_inf']
result_df = pd.DataFrame(columns=columns, dtype=object)
result_df.loc['all'] = get_xresult(results)
result_df

,param,pvalue,stderr,conf_inf
all,-0.010008,4.794654e-62,0.000602,"[-0.01118756921483927, -0.008827568776882597]"


In [195]:
grouped = gss.groupby(gvarname)

for name, group in grouped:
    print(name, len(group))

Catholic 16501
None 7803
Other 2966
Other Christian 896
Protestant 36378


In [196]:
def valid_group(group, yvarname):

    # make sure we have at least 100 values
    num_valid = group[yvarname].notnull().sum()
    if num_valid < 100:
        return False
    
    # make sure all the answers aren't the same
    counts = group[yvarname].value_counts()
    most_common = counts.max()
    
    nonplurality = num_valid - most_common
    if nonplurality < 20:
        return False
        
    return True

In [197]:
for name, group in grouped:

    if valid_group(group, yvarname):
        print(name)

Other Christian
Protestant


In [198]:
for name, group in grouped:
    if not valid_group(group, yvarname):
        continue
    print(group[yvarname].value_counts())
    results = smf.logit(formula, data=group).fit(disp=False)
    result_df.loc[name] = get_xresult(results)
    print(name, results.params['x'])

2.0    633
1.0    107
3.0     97
Name: fund, dtype: int64
Other Christian 0.3483233869681431
1.0    18678
2.0     9856
3.0     7401
Name: fund, dtype: int64
Protestant -0.01632941103830114


In [199]:
result_df

,param,pvalue,stderr,conf_inf
all,-0.010008,4.794654e-62,0.000602,"[-0.01118756921483927, -0.008827568776882597]"
Other Christian,0.348323,7.744273e-39,0.026722,"[0.2959486113167891, 0.40069816261949703]"
Protestant,-0.016329,7.699628e-72,0.000911,"[-0.018115036430687938, -0.01454378564591434]"


In [200]:
def run_subgroups(gss, xvarname, yvarname, gvarname):
    yvar = gss[yvarname]
    counts = yvar.value_counts()
    most_common = counts.idxmax()

    d = counts.copy()
    d[:] = 0
    d[most_common] = 1

    gss['y'] = yvar.replace(d)
    gss['x'] = gss[xvarname]

    formula = 'y ~ x'
    results = smf.logit(formula, data=gss).fit(disp=False)

    param = results.params['x']
    pvalue = results.pvalues['x']
    conf_int = results.conf_int().loc['x'].values
    stderr = results.bse['x']

    columns = ['param', 'pvalue', 'stderr', 'conf_inf']
    result_df = pd.DataFrame(columns=columns, dtype=object)
    result_df.loc['all'] = get_xresult(results)

    grouped = gss.groupby(gvarname)
    for name, group in grouped:
        if not valid_group(group, yvarname):
            continue
    
        results = smf.logit(formula, data=group).fit(disp=False)
        result_df.loc[name] = get_xresult(results)

    return result_df

In [201]:
xvarname = 'year'
yvarname = 'gunlaw'
gvarname = 'partyid4'

result_df = run_subgroups(gss, xvarname, yvarname, gvarname)
result_df

,param,pvalue,stderr,conf_inf
all,0.003234,6.733636e-05,0.000812,"[0.0016437864240885886, 0.0048249216360200945]"
Democrat,0.019363,2.944667e-38,0.001497,"[0.01642815207928425, 0.022297010084502907]"
Independent,0.002507,6.016695e-02,0.001334,"[-0.00010722608201531704, 0.005120890341326177]"
Other,-0.013315,7.678432e-03,0.004995,"[-0.02310464633097545, -0.0035258814981323633]"
Republican,-0.009366,3.977877e-09,0.001592,"[-0.012485542444758965, -0.006246924054176914]"


## Get Differences


In [202]:
stderr = result_df.loc['all', 'stderr']
stderr

0.0008115289967121576

In [203]:
params = result_df['param'].values[1:]
a = np.abs(np.subtract.outer(params, params)) / stderr
a

array([[ 0.        , 20.77035943, 40.26700849, 35.40084759],
       [20.77035943,  0.        , 19.49664905, 14.63048816],
       [40.26700849, 19.49664905,  0.        ,  4.86616089],
       [35.40084759, 14.63048816,  4.86616089,  0.        ]])

In [204]:
names = result_df.index.values[1:]
diff_df = pd.DataFrame(a, index=names, columns=names).stack()
diff_df

Democrat     Democrat        0.000000
             Independent    20.770359
             Other          40.267008
             Republican     35.400848
Independent  Democrat       20.770359
             Independent     0.000000
             Other          19.496649
             Republican     14.630488
Other        Democrat       40.267008
             Independent    19.496649
             Other           0.000000
             Republican      4.866161
Republican   Democrat       35.400848
             Independent    14.630488
             Other           4.866161
             Republican      0.000000
dtype: float64

In [205]:
diff_df.max(), diff_df.idxmax()

(40.2670084850191, ('Democrat', 'Other'))

In [206]:
def get_differences(result_df):
    stderr = result_df.loc['all', 'stderr']
    params = result_df['param'].values[1:]
    a = np.abs(np.subtract.outer(params, params)) / stderr
    names = result_df.index.values[1:]
    diff_df = pd.DataFrame(a, index=names, columns=names).stack()
    return diff_df

In [207]:
diff_df = get_differences(result_df)

In [208]:
diff_df.max(), diff_df.idxmax()

(40.2670084850191, ('Democrat', 'Other'))

In [209]:
def check_simpson(result_df):
    param_all = result_df['param'].values[0]
    params = result_df['param'].values[1:]
    return (param_all < params.min()) or (param_all > params.max())

In [210]:
check_simpson(result_df)

False

In [211]:
yvarnames = ['abany', 'abdefect', 'abhlth', 'abnomore', 'abpoor', 'abrape', 
            'absingle', 'affrmact', 'bible', 'cappun', 'colath', 'colcom', 
            'colhomo', 'colmil', 'colmslm', 'colrac', 'compuse', 'conarmy', 
            'conbus', 'conclerg', 'coneduc', 'confed', 'confinan', 'conjudge', 
            'conlabor', 'conlegis', 'conmedic', 'conpress', 'consci', 'contv', 
            'databank', 'discaffm', 'discaffw', 'divlaw', 'divorce', 'fair', 
            'fear', 'fechld', 'fefam', 'fehire', 'fejobaff', 'fepol', 'fepresch', 
            'finrela', 'fund', 'fund16', 'god', 'grass', 'gunlaw', 'hapmar', 
            'happy', 'health', 'helpful', 'homosex', 'hunt', 'letdie1', 
            'libath', 'libcom', 'libhomo', 'libmil', 'libmslm', 'librac', 
            'life', 'memchurh', 'meovrwrk', 'nataid', 'natarms', 'natchld', 
            'natcity', 'natcrime', 'natcrimy', 'natdrug', 'nateduc', 'natenrgy', 
            'natenvir', 'natfare', 'natheal', 'natmass', 'natpark', 'natrace', 
            'natroad', 'natsci', 'natsoc', 'natspac', 'pornlaw', 
            'postlife', 'pray', 'prayer', 'premarsx', 'pres04', 'pres08', 'pres12', 
            'racmar', 'racopen', 'racpres', 'reborn', 'relexp', 'reliten', 
            'relpersn', 'res16', 'satfin', 'satjob', 'savesoul', 'sexeduc', 
            'spanking', 'spkath', 'spkcom', 'spkhomo', 'spkmil', 'spkmslm', 
            'spkrac', 'sprtprsn', 'teensex', 'trust', 'union_', 'xmarsex',
            'owngun', 'pistol', 'attend', 'relactiv']

len(yvarnames)

120

In [212]:
xvarname = 'year'
gvarname = 'polviews3'

for yvarname in yvarnames:
    result_df = run_subgroups(gss, xvarname, yvarname, gvarname)
    if check_simpson(result_df):
        print(xvarname, yvarname, gvarname)

year finrela polviews3
year happy polviews3
year pres04 polviews3
year pres08 polviews3
year spkath polviews3


In [132]:
gvarnames = ['sex', 'race', 'polviews3', 'partyid4', 'relig5', 
             'age5', 'year5', 'cohort10', 'class_', 'degree']

In [ ]:
xvarname = 'year'

for yvarname in yvarnames:
    for gvarname in gvarnames:
        print(xvarname, yvarname, gvarname)
        if xvarname == 'year' and gvarname == 'year5':
            continue
        result_df = run_subgroups(gss, xvarname, yvarname, gvarname)
        if check_simpson(result_df):
            print(xvarname, yvarname, gvarname)

year abany sex
year abany race
year abany polviews3
year abany partyid4
year abany relig5
year abany age5
year abany year5
year abany cohort10
year abany class_
year abany degree
year abany degree
year abdefect sex
year abdefect race
year abdefect polviews3
year abdefect partyid4
year abdefect relig5
year abdefect age5
year abdefect year5
year abdefect cohort10
year abdefect class_
year abdefect degree
year abhlth sex
year abhlth race
year abhlth polviews3
year abhlth partyid4
year abhlth relig5
year abhlth age5
year abhlth year5
year abhlth cohort10
year abhlth class_
year abhlth degree
year abnomore sex
year abnomore race
year abnomore polviews3
year abnomore partyid4
year abnomore relig5


In [98]:
xvarname = 'year'
yvarname = 'affrmact'
gvarname = 'cohort10'

result_df = run_subgroups(gss, xvarname, yvarname, gvarname)

ValueError: zero-size array to reduction operation maximum which has no identity

In [77]:
check_simpson(result_df)

True

In [78]:
result_df

,param,pvalue,stderr,conf_inf
all,0.019594,4.247140e-127,0.000817,"[0.01799297243019514, 0.02119567114072278]"
Conservative,0.023169,2.256424e-47,0.001603,"[0.020027584106472016, 0.026309484259065864]"
Liberal,0.020634,2.746705e-26,0.001945,"[0.016821556711962532, 0.02444666667461578]"
Moderate,0.021426,1.247678e-48,0.001462,"[0.018560240070909055, 0.02429110942676526]"


In [56]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (9, 4)

def decorate(**options):
    """Decorate the current axes.
    
    Call decorate with keyword arguments like
    decorate(title='Title',
             xlabel='x',
             ylabel='y')
             
    The keyword arguments can be any of the axis properties
    https://matplotlib.org/api/axes_api.html
    """
    ax = plt.gca()
    ax.set(**options)
    
    handles, labels = ax.get_legend_handles_labels()
    if handles:
        ax.legend(handles, labels)

    plt.tight_layout()

In [57]:
def stretch_x(factor = 0.03):
    low, high = plt.xlim()
    space = (high-low) * factor
    plt.xlim(low - space, high + space)

In [58]:
def anchor_legend(x, y):
    """Place the upper left corner of the legend box.
    
    x: x coordinate
    y: y coordinate
    """
    plt.legend(bbox_to_anchor=(x, y), loc='upper left', ncol=1)   
    plt.tight_layout()

In [59]:
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess

def make_lowess(series):
    """Use LOWESS to compute a smooth line.
    
    series: pd.Series
    
    returns: pd.Series
    """
    y = series.values
    x = series.index.values

    smooth = lowess(y, x)
    index, data = np.transpose(smooth)

    return pd.Series(data, index=series.index) 

In [60]:
def plot_series_lowess(series, color, indexed=False, plot_series=True):
    """Plots a series of data points and a smooth line.
    
    series: pd.Series
    color: string or tuple
    """
    if plot_series:
        series.plot(linewidth=0, marker='o', color=color, alpha=0.1, label='_')
    smooth = make_lowess(series)
    if indexed:
        smooth /= smooth.iloc[0] / 100
        
    style = '--' if series.name=='Total, all educational levels' else '-'
    smooth.plot(style=style, label=series.name, color=color)

In [61]:
def plot_columns_lowess(table, columns, colors, **options):
    """Plot the columns in a DataFrame.
    
    table: DataFrame with a cross tabulation
    columns: list of column names, in the desired order
    colors: mapping from column names to colors
    """
    for col in columns:
        series = table[col]
        plot_series_lowess(series, colors[col], **options)

In [62]:
colors = {
    'Total, all educational levels' : 'gray',
    'Less than a high school diploma' : 'C0',
    'High school graduates, no college' : 'C1',
    'Some college or associate degree' : 'C2',
    "Bachelor's degree only" : 'C3',
    'Advanced degree' : 'C4',
}

In [63]:
yvar = gss[yvarname]
counts = yvar.value_counts()
most_common = counts.idxmax()

d = counts.copy()
d[:] = 0
d[most_common] = 1

gss['y'] = yvar.replace(d)
gss['x'] = gss[xvarname]

formula = 'y ~ x'
results = smf.logit(formula, data=gss).fit(disp=False)

param = results.params['x']
pvalue = results.pvalues['x']
conf_int = results.conf_int().loc['x'].values
stderr = results.bse['x']

columns = ['param', 'pvalue', 'stderr', 'conf_inf']
result_df = pd.DataFrame(columns=columns, dtype=object)
result_df.loc['all'] = get_xresult(results)

grouped = gss.groupby(gvarname)
for name, group in grouped:
    results = smf.logit(formula, data=group).fit(disp=False)
    result_df.loc[name] = get_xresult(results)